In [1]:
import sys
import os
src_dir = os.path.join('/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/', 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [2]:
es_client = Elasticsearch("http://localhost:9200")

In [4]:
#es_client.info().body

In [5]:
collected, questions =  read_urls_questions('/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/notebooks/notebooks-clean/URL_Collection/reference_urls_collected.csv',
                                            '/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/notebooks/notebooks-clean/URL_Collection/questions_updated_urls.csv',
                                            clean_url_nan=True) # read collected urls and questions + remove unsuccessful collection

In [6]:
question_list = get_questions(questions)
document_list = get_url_content_tuples(collected)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [7]:
es_client = set_index(es_client=es_client, collected=collected, mappings=mappings)


In [8]:
results = get_result_tuples(es_client=es_client, questions=question_list)

In [9]:
from retrieval.evaluation.evaluate import calculate_recall_at_k, calculate_average_recall

In [10]:
relevant_docs = get_relevant_docs(questions)
recalls = calculate_recall_at_k(question_list, results, relevant_docs, [1, 5, 10], 'bm25') # think of a metric similarity recall? 
calculate_average_recall(recalls)

{1: 0.08823529411764706, 5: 0.3602941176470588, 10: 0.5943627450980393}